In [134]:
!pip -q install folium


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [135]:
import pandas as pd
import numpy as np
import folium
import branca.colormap as cm
from IPython.display import HTML
import psycopg2
from folium.plugins import MousePosition 
import warnings
from folium import Popup
import os
from dotenv import load_dotenv
warnings.filterwarnings('ignore')

In [136]:
conn = psycopg2.connect(
    host=os.getenv("DB_HOST"),
    port=os.getenv("DB_PORT"),
    dbname=os.getenv("DB_NAME"),
    user=os.getenv("DB_USER"),
    password=os.getenv("DB_PASS")
)

In [137]:
data_collar = pd.read_sql("SELECT * FROM data_collar", conn)
data_assay = pd.read_sql("SELECT * FROM data_assay", conn)
data_litologi = pd.read_sql("SELECT * FROM data_litologi", conn)

conn.close()

In [138]:
data_collar['hole_id'] = data_collar['hole_id'].str.extract('(\d+)', expand=False).astype(int)
data_assay['hole_id'] = data_assay['hole_id'].str.extract('(\d+)', expand=False).astype(int)
data_litologi['hole_id'] = data_litologi['hole_id'].str.extract('(\d+)', expand=False).astype(int)

In [139]:
new_data = pd.concat([
    data_assay[['hole_id']],
    data_litologi[['depth_from', 'depth_to', 'litho']],
    data_assay[['ni']]
], axis=1)

# save csv
new_data.to_csv('datasets/new_data.csv', index=False)



In [140]:
sap_data = new_data[new_data['litho'] == 'SAP'].copy()
sap_data['tebal_per_sampel'] = sap_data['depth_to'] - sap_data['depth_from']
sap_data.to_csv('datasets/sap_data.csv', index=False)

In [146]:
sap_data['weighted_ni'] = sap_data['ni'] * sap_data['tebal_per_sampel']

In [145]:
aggregated_data = sap_data.groupby('hole_id').agg(
    depth_from = ('depth_from', 'min'),
    depth_to = ('depth_to', 'max'),
    total_sap = ('tebal_per_sampel', 'sum'),
    min_ni = ('ni', 'min'),
    max_ni = ('ni', 'max')
).reset_index()

In [144]:
weighted_sum = sap_data.groupby('hole_id')['weighted_ni'].sum().reset_index(name='weighted_sum_ni')

In [148]:
aggregated_data = pd.merge(aggregated_data, weighted_sum, on='hole_id')

In [149]:
aggregated_data['avg_ni'] = aggregated_data['weighted_sum_ni'] / aggregated_data['total_sap']

In [150]:
aggregated_data.drop(columns=['weighted_sum_ni'], inplace=True)
aggregated_data.to_csv('datasets/aggregated_data.csv', index=False)

In [154]:
# Define bounds for map view
min_lon, max_lon = plot_data['x'].min(), plot_data['x'].max()
min_lat, max_lat = plot_data['y'].min(), plot_data['y'].max()

# Define value ranges for color mapping
min_avg_ni = plot_data['avg_ni'].min()
max_avg_ni = plot_data['avg_ni'].max()

# Create base map with Esri
map = folium.Map(
    zoom_start=17,
    min_zoom=10,
    control_scale= True,
    font_size='1.5rem',
    tiles="https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}",
    attr='Tiles © Esri — Source: Esri'
)

# Fitting map to data bounds
map.fit_bounds([[min_lat, min_lon], [max_lat, max_lon]])

# Create color scale
colormap = cm.linear.YlOrRd_09.scale(min_avg_ni, max_avg_ni)
colormap.caption = ' Avg. Nickel Value Scale'
colormap.add_to(map)

# Custom CSS for colormap legend background
custom_css = """
<style>
.legend {
    background-color: rgba(255, 255, 255, 0.85);
    padding: 8px 12px;
    border-radius: 6px;
    box-shadow: 0 2px 6px rgba(0,0,0,0.3);
    font-size: 13px;
}
</style>
"""
map.get_root().header.add_child(folium.Element(custom_css))

# Add mouse position to bottom right of map
mouse_position = MousePosition(
    position='bottomright',
    separator=' : ',
    prefix='Lat | Lon:',
    lat_formatter="function(num) {return L.Util.formatNum(num, 10);}",
    lng_formatter="function(num) {return L.Util.formatNum(num, 10);}",
)
mouse_position.add_to(map)

# Add circle markers for each row
for _, row in plot_data.iterrows():
    # color for the circle
    color = colormap(row['avg_ni'])
    # radius size of each circle
    radius = 2 + row['total_sap'] * 0.7

    folium.CircleMarker(
        location=[row['y'], row['x']],
        radius=radius,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=1,
        tooltip=f"<b>BH:</b> {row['hole_id']:.0f}<br>",
        popup=Popup(
        html=(
            f"<b>BH:</b> {row['hole_id']:.0f}<br>"
            f"<b>Lat:</b> {row['x']:.5f}<br>"
            f"<b>Lon:</b> {row['y']:.5f}<br>"
            f"<b>SAP:</b> {row['total_sap']:.2f} meter<br>"
            f"<b>Avg. Ni:</b> {row['avg_ni']:.2f}<br>"
            f"<b>Min. Ni:</b> {row['min_ni']:.2f}<br>"
            f"<b>Max. Ni:</b> {row['max_ni']:.2f}<br>"
        ),
        max_width=200  
    )
    ).add_to(map)

# Displaying the map
# The map will fill the entire output cell, so it's customized through css
display(HTML('<div style="width: 1000px; height: 700px;">'+map._repr_html_()+'</div>'))
# map # uncomment this to use show full size of the map

In [182]:
# make grid
gridx = np.linspace(plot_data['x'].min(), plot_data['x'].max(), 500)
gridy = np.linspace(plot_data['y'].min(), plot_data['y'].max(), 500)
xi, yi = np.meshgrid(gridx, gridy)
xi

array([[121.5056048 , 121.50562095, 121.50563711, ..., 121.51363309,
        121.51364925, 121.5136654 ],
       [121.5056048 , 121.50562095, 121.50563711, ..., 121.51363309,
        121.51364925, 121.5136654 ],
       [121.5056048 , 121.50562095, 121.50563711, ..., 121.51363309,
        121.51364925, 121.5136654 ],
       ...,
       [121.5056048 , 121.50562095, 121.50563711, ..., 121.51363309,
        121.51364925, 121.5136654 ],
       [121.5056048 , 121.50562095, 121.50563711, ..., 121.51363309,
        121.51364925, 121.5136654 ],
       [121.5056048 , 121.50562095, 121.50563711, ..., 121.51363309,
        121.51364925, 121.5136654 ]], shape=(500, 500))

In [183]:
from pykrige.ok import OrdinaryKriging

# define value x, y, z (weighted avg ni) for interpolation
x = plot_data['x'].values
y = plot_data['y'].values
z = plot_data['avg_ni'].values

# interpolate x, y, z
OK = OrdinaryKriging(
    x, y, z,
    variogram_model= "hole-effect", # linear, gaussian, spherical, exponential, hole-effect
    verbose= False,
    enable_plotting= False
)

# interpolate to grid
zi, ss = OK.execute('grid', gridx, gridy)

In [184]:
"""""
Make interpolation map with plotly, because it supports contour map
"""
import plotly.graph_objects as go 

fig = go.Figure()

# buat contour / heatmap
fig.add_trace(go.Contour(
    z= zi,
    x= gridx,
    y= gridy,
    colorscale= 'Gray',
    contours= dict(showlabels= True),
    colorbar= dict(title= 'Avg Ni Scale')
))

# Buat titik borehole
fig.add_trace(go.Scatter(
    x= plot_data['x'],
    y= plot_data['y'],
    mode= 'markers+text',
    text= [f"{v:.2f}" for v in plot_data['avg_ni']],
    textposition= 'top center', 
    marker= dict(color= 'red', size= 8, line= dict(color= 'black', width= 1)),
    name= 'Borehole'
))

fig.update_layout(
    title= 'Interpolated Borehole Map',
    xaxis_title= 'x',
    yaxis_title= 'y',
    width= 1400,
    height= 1000
)

fig.show()